In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
transfer_filt = Filter.create_filter(address=None, event_types=[Pair.events.Transfer])  # Listen events from any smart contract

In [4]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=transfer_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])

    if(len(arguments) >= 1):
        event['details']['transfer_value'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,958,084 tx:0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc476c64fce0fb70287cf4
Transfer at block:60,958,084 tx:0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc476c64fce0fb70287cf4
Transfer at block:60,958,084 tx:0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc476c64fce0fb70287cf4
Transfer at block:60,958,084 tx:0x23601ff0850541ddb910d4c34aacb6d1eb9dedca36ec992ac7d5bbea5e758bc3
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer at block:60,958,084 tx:0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e26c8913613840cb4616f8
Transfer a

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,transfer,sushi,0x172370d5cd63279efa6d502dab29171933a610af,0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc...,60958084,1724452157,{'web3_type': <class 'web3._utils.datatypes.Tr...
1,polygon,uniswapv2pair,transfer,sushi,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc...,60958084,1724452157,{'web3_type': <class 'web3._utils.datatypes.Tr...
2,polygon,uniswapv2pair,transfer,sushi,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xdcc85ac71c6ed271f95a63da3170884eb6988b2f0bfc...,60958084,1724452157,{'web3_type': <class 'web3._utils.datatypes.Tr...
3,polygon,uniswapv2pair,transfer,sushi,0x3c499c542cef5e3811e1192ce70d8cc03d5c3359,0x23601ff0850541ddb910d4c34aacb6d1eb9dedca36ec...,60958084,1724452157,{'web3_type': <class 'web3._utils.datatypes.Tr...
4,polygon,uniswapv2pair,transfer,sushi,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0xbce5c753f61c3f60e2568c6744d48b9c4e0fda9972e2...,60958084,1724452157,{'web3_type': <class 'web3._utils.datatypes.Tr...
...,...,...,...,...,...,...,...,...,...
801,polygon,uniswapv2pair,transfer,sushi,0x5f2f8818002dc64753daedf4a6cb2ccb757cd220,0x9505dd78c44bb6187db6173f11a46f7616c50fdea461...,60958093,1724452175,{'web3_type': <class 'web3._utils.datatypes.Tr...
802,polygon,uniswapv2pair,transfer,sushi,0x5f2f8818002dc64753daedf4a6cb2ccb757cd220,0x9505dd78c44bb6187db6173f11a46f7616c50fdea461...,60958093,1724452175,{'web3_type': <class 'web3._utils.datatypes.Tr...
803,polygon,uniswapv2pair,transfer,sushi,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,0x9505dd78c44bb6187db6173f11a46f7616c50fdea461...,60958093,1724452175,{'web3_type': <class 'web3._utils.datatypes.Tr...
804,polygon,uniswapv2pair,transfer,sushi,0x65f98aa41afac485730cc7ed9a55782de0451afe,0x3928697d3b288aa03eddc0f8e77a513f2f9b264e680f...,60958093,1724452175,{'web3_type': <class 'web3._utils.datatypes.Tr...
